In [47]:
from io import open
import conllu

with open("../milestone 1/dev_sexism_dataset_conllu.conllu", encoding='ISO-8859-1') as f:
    data = conllu.parse(f.read())

X = []
y = []

for tokenlist in data:
    y.append(tokenlist.metadata['label_sexist'])
    X.append([token['form'] for token in tokenlist])
    

In [42]:
print(X[0])
print(y[0])

['nigeria', ',', 'rape', 'woman', ',', 'men', 'rape', 'back', '!', 'nsfw', '``', 'nigeria', ',', 'rape', 'woman', ',', 'guys', 'rape', 'back', '!', "''"]
not sexist


In [43]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

X = pad_sequences(X_seq, padding='post')
X = np.array(X)
y = np.array([1 if label == 'sexist' else 0 for label in y])

In [45]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7313 - loss: 0.5800
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7483 - loss: 0.5669
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7596 - loss: 0.5527
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7700 - loss: 0.5422
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7288 - loss: 0.5787
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7550 - loss: 0.5146
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7574 - loss: 0.4971
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7617 - loss: 0.4996
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7557 - loss: 0.4974
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7666 - loss: 0.4794


In [60]:
import pydot
import graphviz
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
Image(filename='model_architecture.png')

You must install pydot (`pip install pydot`) for `plot_model` to work.


FileNotFoundError: [Errno 2] No such file or directory: 'model_architecture.png'